In [ ]:
import pandas as pd

# Загрузка TSV файла
df = pd.read_csv("data/external/news_tg_csv/telegram_news.tsv", sep="\t")

# Сохранение в CSV
df.to_csv("data/external/news_tg_csv/telegram_news_2.сsv", sep=",", index=False)

In [ ]:
import csv

input_file = "../data/100000_with_emb.csv"
output_file = "../data/30000_with_emb.csv"

# Открываем входной файл
with open(input_file, "r", newline="", encoding="utf-8") as infile:
    reader = list(csv.reader(infile))  # Читаем все строки в список

# Берем заголовок и последние 10 000 строк
header = reader[0]  # Заголовок (первая строка)
last_5000_rows = reader[-30000:]  # Последние 10 000 строк

# Записываем в новый файл
with open(output_file, "w", newline="", encoding="utf-8") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)  # Записываем заголовок
    writer.writerows(last_5000_rows)  # Записываем данные

print("Готово! Последние 10 000 строк сохранены в", output_file)

In [ ]:
import pandas as pd

# Пусть input.csv — исходный файл, а output.csv — результирующий файл
input_file = 'input.csv'
output_file = 'output.csv'

# Список названий, при наличии которых строка будет удалена
names_to_remove = ["Раньше всех. Ну почти.", "ТАСС", "Интерфакс"]

# Читаем CSV файл
df = pd.read_csv(input_file)

# Фильтруем строки, оставляя только те, у которых в столбце channel_name нет указанных названий
filtered_df = df[~df['channel_name'].isin(names_to_remove)]

# Сохраняем результат в новый CSV файл (без добавления индекса)
filtered_df.to_csv(output_file, index=False)

print(f"Файл '{output_file}' успешно создан!")

In [ ]:
import pandas as pd
df = pd.read_csv("../data/external/news_tg_csv/telegram_news.сsv")

In [ ]:
df['datetime'].dtype

In [ ]:
import pandas as pd
df = pd.read_csv("../data/raw/news_with_emb.csv")
len(df)

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Устанавливаем даты
end_date = datetime.now()
start_date = end_date - timedelta(days=6*365)

# Формируем URL для API ЦБ РФ
url = f'https://cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From={start_date.strftime("%d.%m.%Y")}&UniDbQuery.To={end_date.strftime("%d.%m.%Y")}'

# Получаем страницу и парсим таблицу
response = requests.get(url)
tables = pd.read_html(response.text)

# Выбираем нужную таблицу
key_rate_table = tables[0]

# Сохраняем данные в CSV
key_rate_table.to_csv('key_rate_cbr.csv', index=False)

print("Данные успешно загружены и сохранены в файл 'key_rate_cbr.csv'")

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET

def get_urals_prices_cbr():
    # Формируем даты для запроса
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*10)  # за последние 10 лет
    
    # Формируем URL запроса
    url = "http://www.cbr.ru/scripts/XML_dynamic.asp"
    params = {
        'date_req1': start_date.strftime('%d/%m/%Y'),
        'date_req2': end_date.strftime('%d/%m/%Y'),
        'VAL_NM_RQ': 'R01239'  # Код для нефти Urals
    }
    
    try:
        # Выполняем запрос
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # Парсим XML
        root = ET.fromstring(response.content)
        
        # Собираем данные
        data = []
        for record in root.findall('Record'):
            date = record.get('Date')
            value = record.find('Value').text
            nominal = record.find('Nominal').text
            
            data.append({
                'Date': datetime.strptime(date, '%d.%m.%Y'),
                'Price': float(value.replace(',', '.')),
                'Nominal': int(nominal)
            })
        
        # Создаем DataFrame
        df = pd.DataFrame(data)
        
        # Сортируем по дате
        df = df.sort_values('Date')
        
        # Сохраняем в CSV
        df.to_csv('../../../data/external/macro/urals_prices_cbr.csv', index=False)
        
        print(f"Данные успешно загружены и сохранены в 'urals_prices_cbr.csv'")
        print(f"Количество записей: {len(df)}")
        print("\nПервые 5 записей:")
        print(df.head())
        
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к API ЦБ: {e}")
        return None

if __name__ == "__main__":
    get_urals_prices_cbr()

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def download_brent_prices():
    # Получаем текущую дату
    end_date = datetime.now()
    # Дата 10 лет назад
    start_date = end_date - timedelta(days=365*10)
    
    # Загружаем данные для нефти Brent (тикер BZ=F)
    brent = yf.download('BZ=F', 
                        start=start_date.strftime('%Y-%m-%d'),
                        end=end_date.strftime('%Y-%m-%d'))
    
    # Сохраняем в CSV файл
    brent.to_csv('extra_features/brent_prices_10y.csv')
    
    print(f"Данные сохранены в файл 'brent_prices_10y.csv'")
    print(f"Количество записей: {len(brent)}")
    
    # Показываем первые несколько строк
    print("\nПервые 5 записей:")
    print(brent.head())

if __name__ == "__main__":
    download_brent_prices()

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
# Функция для загрузки данных о курсе доллара за заданный диапазон дат
def get_usd_to_rub_exchange_rate(start_date, end_date):
    base_url = "https://www.cbr.ru/scripts/XML_dynamic.asp"

    # Форматирование дат для запроса
    start_date_str = start_date.strftime('%d/%m/%Y')
    end_date_str = end_date.strftime('%d/%m/%Y')

    # Параметры запроса
    params = {
        'date_req1': start_date_str,
        'date_req2': end_date_str,
        'VAL_NM_RQ': 'R01235'  # Код валюты для USD
    }

    # Запрос к API ЦБ РФ
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        raise Exception(f"Ошибка при запросе данных: {response.status_code}")

    # Парсинг ответа XML
    data = pd.read_xml(response.content, xpath="//Record")

    # Преобразование данных в DataFrame
    data['Date'] = pd.to_datetime(data['Date'], format='%d.%m.%Y')
    data['Value'] = data['Value'].str.replace(',', '.').astype(float)

    # Переименование столбцов для удобства
    data = data.rename(columns={
        'Date': 'Дата',
        'Value': 'Курс USD к RUB'
    })

    return data

# Получение данных за последние 10 лет
end_date = datetime.now()
start_date = end_date - timedelta(days=10*365)

# Загрузка данных
try:
    exchange_rate_data = get_usd_to_rub_exchange_rate(start_date, end_date)
    print("Данные успешно загружены!")
    
    # Сохранение данных в CSV
    output_file = "../../../data/external/macro/usd_to_rub.csv"
    exchange_rate_data.to_csv(output_file, index=False)
    print("Данные сохранены в файл 'usd_to_rub.csv'")
except Exception as e:
    print(f"Произошла ошибка: {e}")

In [ ]:
import pandas as pd
# Загрузка файла с эмбеддингами
emb_file = "../data/external/text/news_with_emb.csv"
df = pd.read_csv(emb_file)

# Поиск колонки с эмбеддингами (предполагаем, что это единственная колонка с JSON-строками)
emb_cols = df.columns[df.apply(lambda x: x.dtype == 'object' and x.str.contains(r'^\[.*\]$').any())]

if len(emb_cols) == 1:
    # Переименование колонки
    old_name = emb_cols[0]
    df = df.rename(columns={old_name: 'embedding'})
    
    # Сохранение обновленного файла
    df.to_csv(emb_file, index=False)
    print(f"Колонка '{old_name}' успешно переименована в 'embedding'")
else:
    print("Не удалось однозначно определить колонку с эмбеддингами")


In [ ]:
import requests
import pandas as pd

def get_moex_listed_stocks():
    """
    Fetches a list of actively traded stocks (shares) from the Moscow Exchange API (TQBR board).

    Returns:
        pandas.DataFrame: DataFrame containing the Ticker (SECID) and Short Name (SHORTNAME)
                          of the listed stocks, or None if an error occurs.
    """
    # MOEX ISS API endpoint for securities traded on the TQBR board (main stock board)
    url = "https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json"
    
    params = {
        "iss.meta": "off",          # Turn off metadata block
        "iss.only": "securities",   # Get only the securities data block
        "securities.columns": "SECID,SHORTNAME,SECNAME,ISIN,PREVADMITTEDQUOTE" # Specify desired columns
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        
        data = response.json()
        
        # Extract securities data
        securities_data = data['securities']['data']
        # Extract column names
        columns = data['securities']['columns']
        
        # Create DataFrame
        df = pd.DataFrame(securities_data, columns=columns)
        
        # Filter out securities that are not currently admitted to trading (optional, but good practice)
        # PREVADMITTEDQUOTE is often the last traded price, non-null for traded stocks
        # A more robust filter might be needed depending on exact MOEX API nuances
        df_traded = df[df['PREVADMITTEDQUOTE'].notna()].copy()

        # Select relevant columns for the output
        df_result = df_traded[['SECID', 'SHORTNAME']].reset_index(drop=True)
        
        return df_result

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса к MOEX API: {e}")
        return None
    except KeyError as e:
        print(f"Ошибка: Неожиданная структура ответа от MOEX API. Отсутствует ключ: {e}")
        return None
    except Exception as e:
        print(f"Произошла непредвиденная ошибка: {e}")
        return None

# --- Execution ---
print("Запрос списка акций с Московской Биржи (board TQBR)...")
stocks_df = get_moex_listed_stocks()

if stocks_df is not None:
    print(f"\nНайдено {len(stocks_df)} торгуемых акций на TQBR:")
    # Display the first few and last few rows for brevity
    with pd.option_context('display.max_rows', 10):
        print(stocks_df)
else:
    print("Не удалось получить список акций.")



In [ ]:
import json, requests, collections, time

FILE_IN  = '../configs/companies_config.json'
FILE_OUT = '../configs/all_companies_config.json'

# 1. вытаскиваем все бумаги с борда TQBR
url = ('https://iss.moex.com/iss/engines/stock/markets/shares/'
       'boards/TQBR/securities.json?iss.meta=off&iss.only=securities')
data = requests.get(url, timeout=10).json()['securities']['data']
cols = requests.get(url.replace('iss.only=securities','iss.meta=on')
                    ).json()['securities']['columns']
col = {name: idx for idx, name in enumerate(cols)}

full_list = {
    row[col['SECID']]: {
        "short": row[col['SHORTNAME']],
        "full":  row[col['SECNAME']]
    } for row in data
}

# 2. загружаем ваш файл
with open(FILE_IN, encoding='utf‑8') as f:
    config = json.load(f)

have = set(config['companies'])

import re

# ── вспомогательная функция ──────────────────────────────────────────────────
_rm_quotes = re.compile(r'[\"«»]')          # удаляем: " « »
def clean(text: str) -> str:
    return _rm_quotes.sub('', text).strip()

# ── 3. добавляем недостающие бумаги в файл конфигурации ───────────────────────
added = 0
for secid, names in full_list.items():
    if secid in have:
        continue

    short = clean(names["short"])
    full  = clean(names["full"])

    config['companies'][secid] = {
        "names": [
            short,
            full,
            secid      # сам тикер (без изменений)
        ]
    }
    added += 1

# 4. сохраняем
with open(FILE_OUT, 'w', encoding='utf‑8') as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print(f'Добавлено {added} новых компаний → {FILE_OUT}')

In [ ]:
import tiktoken
import json

# Выберите нужную модель
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")  

total_tokens = 0

with open("../data/external/text/batch/batch_input_example.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        # Предположим, что текст находится в поле "text"
        text = data.get("text", "")
        tokens = encoding.encode(text)
        total_tokens += len(tokens)

print(f"Total tokens: {total_tokens}")

In [ ]:
pip install requests beautifulsoup4 wikipedia

In [ ]:
import json, re, requests, wikipedia, bs4, time
wikipedia.set_lang("ru")

def fetch_aliases(base_name):
    try:
        page = wikipedia.page(base_name, auto_suggest=False, redirect=True)
        soup = bs4.BeautifulSoup(requests.get(page.url).text, "html.parser")
        first_p = soup.select_one("p").get_text(" ", strip=True)
    except Exception:
        return []

    # простая эвристика: берём все строки в кавычках/скобках
    raw = re.findall(r"[«\"“](.+?)[»\"”)]", first_p) + \
          re.findall(r"\(([^)]+)\)", first_p)
    # подчистим
    return {re.sub(r"\s+", " ", a).strip() for a in raw if 3 < len(a) < 80}

with open("../configs/all_companies_config.json", encoding="utf-8") as f:
    cfg = json.load(f)

for ticker, info in cfg["companies"].items():
    base = info["names"][0]
    aliases = fetch_aliases(base)
    cfg["companies"][ticker]["names"] = sorted(set(info["names"]) | aliases)
    time.sleep(0.5)          # не злоупотребляем API

with open("../configs/all_companies_config_enriched.json", "w", encoding="utf-8") as f:
    json.dump(cfg, f, ensure_ascii=False, indent=2)

In [ ]:
import json
import re
from pathlib import Path

# Файлы‑источники
ALL = Path('../configs/all_companies_config.json')
CUR = Path('../configs/companies_config.json')

# Регулярка: «ПАО», «АО», «ао» – независимо от регистра, как отдельное слово
rm_re = re.compile(r'\b(?:п?ао)\b', re.IGNORECASE | re.UNICODE)

# Загружаем оба конфигурационных файла
with ALL.open(encoding='utf‑8') as f:
    all_cfg = json.load(f)

with CUR.open(encoding='utf‑8') as f:
    cur_cfg = json.load(f)

# Тикеры, которых нет во втором файле
extra_tickers = set(all_cfg["companies"]) - set(cur_cfg["companies"])

for ticker in extra_tickers:
    names = all_cfg["companies"][ticker]["names"]
    new_names = []
    for name in names:
        # удаляем «ПАО»/«АО»/«ао»
        clean = rm_re.sub('', name)
        # убираем возможные двойные пробелы и обрезаем края
        clean = re.sub(r'\s{2,}', ' ', clean).strip()
        # добавляем пробелы в начале и в конце
        new_names.append(f'{clean}')
    all_cfg["companies"][ticker]["names"] = new_names

# Сохраняем результат в новый файл (можно перезаписать исходный, если нужно)
out = ALL.with_name('all_companies_config.json')
with out.open('w', encoding='utf‑8') as f:
    json.dump(all_cfg, f, ensure_ascii=False, indent=2)

print(f'Готово! Сохранено в {out}')

In [ ]:
import pandas as pd
import json
import numpy as np

# Пути к файлам
config_file_path = '../configs/all_companies_config.json'
input_csv_path = '../data/processed/gpt/last_20_rows.csv' # Убедитесь, что это правильный путь к вашему файлу
output_csv_path = '../data/processed/gpt/last_20_rows_cleaned.csv'

try:
    # 1. Загрузка конфигурации
    with open(config_file_path, 'r', encoding='utf-8') as f:
        config_data = json.load(f)
    print(f"Конфигурационный файл '{config_file_path}' успешно загружен.")

    # 2. Загрузка CSV
    try:
        df = pd.read_csv(input_csv_path)
        print(f"CSV файл '{input_csv_path}' успешно загружен. Количество строк: {len(df)}, количество колонок: {len(df.columns)}")
    except FileNotFoundError:
        print(f"Ошибка: CSV файл '{input_csv_path}' не найден.")
        exit()
    except Exception as e:
        print(f"Ошибка при чтении CSV файла '{input_csv_path}': {e}")
        exit()

    # 3. Создание карты сопоставления (название -> канонический тикер)
    name_to_canonical_map = {}
    # Компании
    for canonical_ticker, company_info in config_data.get('companies', {}).items():
        name_to_canonical_map[canonical_ticker.lower()] = canonical_ticker
        for name_variant in company_info.get('names', []):
            name_to_canonical_map[name_variant.lower()] = canonical_ticker
    # Индексы
    for canonical_ticker, description in config_data.get('indices', {}).items():
        name_to_canonical_map[canonical_ticker.lower()] = canonical_ticker
        # name_to_canonical_map[description.lower()] = canonical_ticker # Раскомментируйте, если описания тоже могут быть именами колонок

    print(f"Карта сопоставления создана. Уникальных канонических тикеров в карте: {len(set(name_to_canonical_map.values()))}")

    # Колонки, которые нужно сохранить без изменений
    preserved_columns = ['date', 'summary']
    cleaned_df_columns = {} # Словарь для сбора колонок нового DataFrame

    for col in preserved_columns:
        if col in df.columns:
            cleaned_df_columns[col] = df[col]
        else:
            print(f"Предупреждение: Ожидаемая колонка '{col}' не найдена в CSV.")

    # 4. Идентификация и группировка колонок
    column_groups = {}  # {'SBER': ['SBER', 'Sberbank_typo'], ...}
    
    # Определяем, какие из исходных колонок являются числовыми (кроме preserved_columns)
    potential_instrument_cols = [col for col in df.columns if col not in preserved_columns]
    
    for original_col_name in potential_instrument_cols:
        # Попытка приведения к нижнему регистру, если это строка
        try:
            col_name_lower = str(original_col_name).lower()
        except Exception: # На случай если имя колонки не может быть строкой (маловероятно для CSV)
            col_name_lower = original_col_name

        canonical_ticker = name_to_canonical_map.get(col_name_lower)
        
        if canonical_ticker:
            if canonical_ticker not in column_groups:
                column_groups[canonical_ticker] = []
            column_groups[canonical_ticker].append(original_col_name)
        else:
            # Если нужно, можно раскомментировать для отладки ненайденных колонок
            print(f"Информация: Колонка '{original_col_name}' не имеет прямого сопоставления в конфигурации и не будет объединена, если она не числовая или не входит в другую группу.")

    print(f"Найдено {len(column_groups)} групп колонок для объединения.")

    # 5. Агрегация значений
    for canonical_ticker, original_cols_list in column_groups.items():
        # Убедимся, что все колонки в группе существуют в DataFrame
        existing_cols_in_group = [col for col in original_cols_list if col in df.columns]
        
        if not existing_cols_in_group:
            print(f"Предупреждение: Для канонического тикера '{canonical_ticker}' не найдено соответствующих колонок в CSV из списка {original_cols_list}.")
            continue

        # Преобразуем данные в числовой формат, ошибки заменим на NaN
        # Это важно, чтобы .abs() и .idxmax() работали корректно
        numeric_data = df[existing_cols_in_group].apply(pd.to_numeric, errors='coerce')

        def get_max_abs_value(row_series):
            if row_series.isnull().all():
                return np.nan 
            else:
                # .abs() для модуля, .idxmax() для индекса максимального значения в серии модулей
                # .loc[] для получения исходного значения (не модуля) по этому индексу
                return row_series.loc[row_series.abs().idxmax()]

        cleaned_df_columns[canonical_ticker] = numeric_data.apply(get_max_abs_value, axis=1)
        
    # 6. Формирование итогового DataFrame
    final_df = pd.DataFrame(cleaned_df_columns)
    
    # Упорядочивание колонок: сначала 'date', 'summary', затем остальные по алфавиту
    final_cols_order = [col for col in preserved_columns if col in final_df.columns]
    instrument_cols = sorted([col for col in final_df.columns if col not in preserved_columns])
    final_df = final_df[final_cols_order + instrument_cols]

    print(f"Итоговый DataFrame содержит {len(final_df.columns)} колонок.")

    # 7. Сохранение результата
    try:
        final_df.to_csv(output_csv_path, index=False, encoding='utf-8')
        print(f"Очищенный DataFrame успешно сохранен в '{output_csv_path}'")
        print("\nПервые 5 строк очищенного файла:")
        print(final_df.head().to_string())
        print(f"\nПоследние 5 строк очищенного файла:")
        print(final_df.tail().to_string())

    except Exception as e:
        print(f"Ошибка при сохранении итогового CSV файла: {e}")

except FileNotFoundError:
    print(f"Критическая ошибка: Конфигурационный файл '{config_file_path}' не найден. Скрипт не может продолжить работу.")
except json.JSONDecodeError:
    print(f"Критическая ошибка: Не удалось декодировать JSON из файла '{config_file_path}'. Проверьте его структуру. Скрипт не может продолжить работу.")
except Exception as e:
    print(f"Произошла непредвиденная ошибка во время выполнения скрипта: {e}")


In [ ]:
import pandas as pd

# Читаем последние 10 строк из CSV файла
df = pd.read_csv('../data/processed/gpt/results_gpt_blogs.csv', engine='python').tail(10)

# Сохраняем копию в новый файл
output_path = '../data/processed/gpt/results_gpt_blogs_last_10.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"Копия последних 10 строк сохранена в {output_path}")
print("\nСодержимое файла:")
print(df)



In [ ]:
import os
import pandas as pd

# Пути к директориям
merged_features_dir = '../data/processed/merged_features'
features_final_dir = '../data/features_final'

# Получаем списки файлов из обеих директорий
merged_files = set([f.split('_merged.csv')[0] for f in os.listdir(merged_features_dir) if f.endswith('_merged.csv')])
final_files = set([f.split('_final.csv')[0] for f in os.listdir(features_final_dir) if f.endswith('_final.csv')])

# Находим тикеры, которые есть в merged_features, но отсутствуют в features_final
missing_tickers = merged_files - final_files

print(f"Тикеры, которые есть в merged_features, но отсутствуют в features_final:")
for ticker in sorted(missing_tickers):
    print(f"- {ticker}")

print(f"\nВсего найдено {len(missing_tickers)} отсутствующих тикеров")
